# U-Net Segmentation

### Data extraction

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os 
import re
import cv2
import pydicom
import nibabel as nib
from glob import glob
from keras.layers import * 
from keras.models import Model
from keras import backend as K
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
tf.config.run_functions_eagerly(True)

In [ ]:
metadata_path = "/kaggle/input/rsna-2023-abdominal-trauma-detection/train_series_meta.csv"

train_metadata = pd.read_csv(metadata_path)
train_metadata.head()

In [ ]:
segmentations_path = "/kaggle/input/rsna-2023-abdominal-trauma-detection/segmentations"

segmentations = os.listdir(segmentations_path)
segmentations = [int(os.path.splitext(segmentation)[0]) for segmentation in segmentations]

print(f"Total number of series id that are supported with a segmentation file: {len(segmentations)}")

In [ ]:
series = train_metadata["series_id"].tolist()

matched_series = []

for segmentation in segmentations:
    if segmentation in series:
        matched_series.append(segmentation)
    else:
        continue

len(matched_series)

In [ ]:
patients_segment = train_metadata[train_metadata["series_id"].isin(matched_series)].reset_index(drop=True)
patients_with_segmentations = patients_segment["patient_id"].unique()
patients_segment

Now we had mapped successfully the patient id and series id from the training data that we will use for segmentation purpose

### Data cleaning -> DICOM and NIFTII

In [ ]:
def extract_number_from_path(path):
    match = re.search(r'(\d+)\.dcm$', path)
    if match:
        return int(match.group(1))
    return 0

def get_data_for_3d_volumes(data, dcm_path, niftii_path):

    data_to_merge = data[["patient_id", "series_id"]]
    
    total_paths = []
    patient_ids = []
    series_ids = []
    seg_path = []
    
    for patient_id in range(len(data_to_merge)):
    
        p_id = str(data_to_merge["patient_id"][patient_id]) + "/" + str(data_to_merge["series_id"][patient_id])
        str_imgs_path = dcm_path + p_id + '/'
        
        seg_mask_paths = niftii_path + str(data_to_merge["series_id"][patient_id]) + ".nii"
        seg_path.append(seg_mask_paths)
        
        patient_img_paths = []

        for file in glob(str_imgs_path + '/*'):
            patient_img_paths.append(file)
        
        
        sorted_file_paths = sorted(patient_img_paths, key=extract_number_from_path)
        total_paths.append(sorted_file_paths)
        patient_ids.append(data_to_merge["patient_id"][patient_id])
        series_ids.append(data_to_merge["series_id"][patient_id])
    
    final_data = pd.DataFrame(list(zip(patient_ids, series_ids, total_paths, seg_path)),
               columns =["patient_id","series_id", "patient_paths", "patient_segmentation"])
    
    return final_data

In [ ]:
dcm_path = "/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images/"
niftii_path = "/kaggle/input/rsna-2023-abdominal-trauma-detection/segmentations/"

cleaned_data = get_data_for_3d_volumes(patients_segment, dcm_path, niftii_path)

In [ ]:
cleaned_data.head()

In [ ]:
def window_converter(image, window_width=400, window_level=50):      
    img_min = window_level - window_width // 2
    img_max = window_level + window_width // 2
    window_image = image.copy()
    window_image[window_image < img_min] = img_min
    window_image[window_image > img_max] = img_max
    #image = (image / image.max() * 255).astype(np.float64)
    return window_image

def transform_to_hu(medical_image, image):
    meta_image = pydicom.dcmread(medical_image)
    intercept = meta_image.RescaleIntercept
    slope = meta_image.RescaleSlope
    hu_image = image * slope + intercept
    return hu_image

def standardize_pixel_array(dcm: pydicom.dataset.FileDataset) -> np.ndarray:
    # Correct DICOM pixel_array if PixelRepresentation == 1.
        pixel_array = dcm.pixel_array
        if dcm.PixelRepresentation == 1:
            bit_shift = dcm.BitsAllocated - dcm.BitsStored
            dtype = pixel_array.dtype 
            pixel_array = (pixel_array << bit_shift).astype(dtype) >> bit_shift
        return pixel_array

def resize_img(img_paths, target_size=(128, 128)):
        volume_shape = (target_size[0], target_size[1], len(img_paths)) 
        volume = np.zeros(volume_shape, dtype=np.float64)
        for i, image_path in enumerate(img_paths):
            image = pydicom.read_file(image_path)
            image = standardize_pixel_array(image)
            hu_image = transform_to_hu(image_path, image)
            window_image = window_converter(hu_image)
            image = cv2.resize(window_image, target_size)
            volume[:,:,i] = image
        return volume
    
def normalize_volume(resized_volume):
    original_shape = resized_volume.shape
    flattened_image = resized_volume.reshape((-1,))
    scaler = preprocessing.MinMaxScaler()
    normalized_flattened_image = scaler.fit_transform(flattened_image.reshape((-1, 1)))
    normalized_volume_image = normalized_flattened_image.reshape(original_shape)
    return normalized_volume_image

def create_3D_segmentations(filepath, target_size, downsample_rate=1):
    img = nib.load(filepath).get_fdata()
    img = np.transpose(img, [2, 1, 0])
    img = np.rot90(img, -1, (1,2))
    img = img[::-1,:,:]
    img = np.transpose(img, [2, 1, 0])
    img = img[::downsample_rate, ::downsample_rate, ::downsample_rate]
    
    resized_images = []

    for i in range(img.shape[2]):
        resized_img = cv2.resize(img[:, :, i], target_size)
        resized_images.append(resized_img)
    
    resized_3D_mask = np.stack(resized_images, axis=2)
    
    return np.array(resized_3D_mask, dtype=np.int8)

def generate_patient_processed_data(list_img_paths, list_seg_paths, target_size=(128,128)):

    height = target_size[0]
    width = target_size[1]
    depth = len(list_img_paths)

    volume_array = np.zeros((height, width, depth), dtype=np.float64)

    print("Initializing data preprocessing with the following dimensions-> Volumes:{}".format(volume_array.shape))

    resized_images = resize_img(list_img_paths, target_size=target_size)
    normalized_siz_volume = normalize_volume(resized_images)
    volume_array = normalized_siz_volume
    volume_mask = create_3D_segmentations(list_seg_paths, target_size=target_size)
    
    transposed_volume_dcm = np.transpose(volume_array, (2, 0, 1))
    transpose_volume_nii = np.transpose(volume_mask, (2, 0, 1))
    
    labelencoder = LabelEncoder()

    n, h, w = transpose_volume_nii.shape
    train_masks_reshaped = transpose_volume_nii.reshape(-1,1)
    train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
    train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)

    transposed_volume_dcm = np.expand_dims(transposed_volume_dcm, axis=3)
    transpose_volume_nii = np.expand_dims(train_masks_encoded_original_shape, axis=3)

    return transposed_volume_dcm, transpose_volume_nii

In [ ]:
volume_dcm = []
volume_nii = []

for i in range(1):
    volume_img, volume_seg = generate_patient_processed_data(cleaned_data["patient_paths"][i], cleaned_data["patient_segmentation"][i])
    
    volume_dcm.append(volume_img)
    volume_nii.append(volume_seg)

In [ ]:
volume_of_imgs = np.concatenate(volume_dcm, axis=0)
volume_of_segs = np.concatenate(volume_nii, axis=0)
volume_of_imgs.shape, volume_of_segs.shape

In [ ]:
segmentation = create_3D_segmentations(cleaned_data["patient_segmentation"][0], target_size=(128, 128), downsample_rate=1)

transpose_volume_nii = np.transpose(segmentation, (2, 0, 1))

In [ ]:
transpose_volume_nii.shape[0]

In [ ]:
transpose_volume_nii.dtype

In [ ]:
np.unique(transpose_volume_nii)

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = transpose_volume_nii.shape
train_masks_reshaped = transpose_volume_nii.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)

np.unique(train_masks_encoded_original_shape)

In [ ]:
train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis=3)
train_masks_input.shape

In [ ]:
from keras.utils import to_categorical

n_classes=6

train_masks_cat = to_categorical(train_masks_input, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((train_masks_input.shape[0], train_masks_input.shape[1], train_masks_input.shape[2], n_classes))

In [ ]:
y_train_cat.dtype

In [ ]:
volume = []
for i in range(len(y_train_cat)):
    y = y_train_cat[i,:,:,1]
    volume.append(y)

In [ ]:
volume = np.array(volume)

In [ ]:
volume = np.expand_dims(volume, axis=3)

In [ ]:
volume.dtype

In [ ]:
np.unique(volume)

In [ ]:
plt.imshow(volume[300,:,:,:], cmap="jet")

In [ ]:
with open(f'/kaggle/working/X_y_segmentations_data.npy', 'wb') as f:
    np.save(f, volume_of_imgs)
    np.save(f, volume_of_segs)

In [ ]:
with open(f'/kaggle/working/X_y_segmentations_data.npy', 'rb') as f:
    X = np.load(f, allow_pickle=True)
    y = np.load(f, allow_pickle=True)

In [ ]:
X.shape, y.shape

In [ ]:
def segmentation_visualization(volume, volume_seg, slice_dcm):
    
    fig = plt.figure(figsize=(14,14), constrained_layout=True)

    ax1 = fig.add_subplot(131)
    ax1.imshow(volume[slice_dcm,:,:], cmap = 'gray')

    ax2 = fig.add_subplot(132)
    ax2.imshow(volume_seg[slice_dcm,:,:], cmap = 'gray')

    ax3 = fig.add_subplot(133)
    ax3.imshow(volume[slice_dcm,:,:]*np.where(volume_seg[slice_dcm,:,:]>0,1,0), cmap = 'gray')
    ax3.set_title('Overlay of Original and Segmented', fontsize=14)
    plt.show()

### Data Generator

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):

    def __init__(self, X_set: np.array, y_set: np.array, num_classes: int, batch_size: int) -> None:
        """_Initialization of data generator_

        Parameters
        ----------
        X_set : np.array
            _Set of images_
        y_set : _np.array_
            _Set of masks_
        num_classes : _int_
            _Number of classes_
        batch_size : _int_
            _Size of the batch taken from X_set and y_set_
        """
        self.x, self.y = X_set, y_set
        self.classes = num_classes
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)
    
    def __getitem__(self, index):
        """_Get item method for each epoch in training
            and computes categorical values for
            segmentation maks_

        Parameters
        ----------
        index : _int_
            _Integer utilized to return the batches_

        Returns
        -------
        _np.array_
            _Returns the batch from X_set and categorical btach from y_set_
        """
        batch_x = self.x[index * self.batch_size:(index + 1) * self.batch_size]
        batch_y = self.y[index * self.batch_size:(index + 1) * self.batch_size]

        train_masks_cat = to_categorical(batch_y, num_classes=self.classes)
        batch_y_categorical = train_masks_cat.reshape((batch_y.shape[0], batch_y.shape[1], batch_y.shape[2], self.classes))

        return batch_x, batch_y_categorical

In [ ]:
data_gen = DataGenerator(X, y, 6, 32)

In [ ]:
x, y = data_gen[0]

In [ ]:
x.shape, y.shape

### Train & Test split and categorical labels

### Segmentations labels


* i. 0 = background
* ii. 1 = liver
* iii. 2 = spleen
* iv. 3 = left kidney
* v. 4 = right kidney
* vi. 5 = bowel

In [ ]:
segmentation_visualization(volume_of_imgs, volume_of_segs, slice_dcm=200)

In [ ]:
number_classes = len(np.unique(volume_of_segs))

X_train , X_test, y_train, y_test = train_test_split(volume_of_imgs, volume_of_segs, test_size = 0.10, shuffle=True)


train_masks_cat = to_categorical(y_train, num_classes=number_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], number_classes))


test_masks_cat = to_categorical(y_test, num_classes=number_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], number_classes))

In [ ]:
len(X_train), len(X_test)

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

n, h, w, _ = volume_of_segs.shape
train_masks_reshaped = volume_of_segs.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped.ravel())
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)

In [ ]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',
                                                 classes = np.unique(train_masks_reshaped_encoded),
                                                 y = train_masks_reshaped_encoded)

print("Class weights are...:", class_weights)

## Custom loss function

In [ ]:
# Custom metrics and loss function

def dice_coef(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    smooth = 0.0001
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_multilabel(y_true, y_pred, numLabels = 6):
    dice = 0
    weights = [0.17649986,  7.78453571, 41.53978194, 65.20657672, 96.75504125,  6.40743063]
    for index in range(numLabels):
        dice += dice_coef(y_true[:,:,:,index], y_pred[:,:,:,index]) * weights[index]
    return dice/np.sum(weights)

def dice_coef_multilabelloss(y_true, y_pred):
    return 1 - dice_coef_multilabel(y_true, y_pred)

def weightedLoss(originalLossFunc, weightsList):

    def lossFunc(true, pred):
        true = K.cast(true, K.floatx())
        pred = K.cast(pred, K.floatx())

        axis = -1 #if channels last 
          #axis=  1 #if channels first


          #argmax returns the index of the element with the greatest value
          #done in the class axis, it returns the class index    
        classSelectors = K.argmax(true, axis=axis) 
              #if your loss is sparse, use only true as classSelectors

          #considering weights are ordered by class, for each class
          #true(1) if the class index is equal to the weight index 
          #weightsList = tf.cast(weightsList, tf.int64)
        classSelectors = [K.equal(tf.cast(i, tf.int64), tf.cast(classSelectors, tf.int64)) for i in range(len(weightsList))]

          #casting boolean to float for calculations  
          #each tensor in the list contains 1 where ground true class is equal to its index 
          #if you sum all these, you will get a tensor full of ones. 
        classSelectors = [K.cast(x, K.floatx()) for x in classSelectors]

          #for each of the selections above, multiply their respective weight
        weights = [sel * w for sel,w in zip(classSelectors, weightsList)] 

          #sums all the selections
          #result is a tensor with the respective weight for each element in predictions
        weightMultiplier = weights[0]
        for i in range(1, len(weights)):
            weightMultiplier = weightMultiplier + weights[i]


          #make sure your originalLossFunc only collapses the class axis
          #you need the other axes intact to multiply the weights tensor
        loss = originalLossFunc(true,pred) 
        loss = loss * weightMultiplier

        return loss
    return lossFunc

In [ ]:
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
        return K.mean(K.stack(prec), axis=0)

LR = 0.001

def dice_coef(y_true, y_pred):
    smooth=1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def bce_dice_loss(y_true, y_pred):
    return 0.5 * tf.keras.losses.binary_crossentropy(y_true, y_pred) - dice_coef(y_true, y_pred)



## U-Net model architecture

In [ ]:
tf.keras.backend.clear_session()
nb_filter = [32,64,128,256,512]
# Build U-Net++ model
inputs = Input((128, 128, 1))
#s = Lambda(lambda x: x / 255)(inputs)
c1 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(inputs)
c1 = Dropout(0.5)(c1)
c1 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(c1)
c1 = Dropout(0.5)(c1)
p1 = MaxPooling2D((2, 2), strides=(2, 2))(c1)

c2 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(p1)
c2 = Dropout(0.5)(c2)
c2 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(c2)
c2 = Dropout(0.5)(c2)
p2 = MaxPooling2D((2, 2), strides=(2, 2))(c2)

up1_2 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up12', padding="same")(c2)
conv1_2 = concatenate([up1_2, c1], name='merge12', axis=3)
c3 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(conv1_2)
c3 = Dropout(0.5)(c3)
c3 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(c3)
c3 = Dropout(0.5)(c3)

conv3_1 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(p2)
conv3_1 = Dropout(0.5)(conv3_1)
conv3_1 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(conv3_1)
conv3_1 = Dropout(0.5)(conv3_1)
pool3 = MaxPooling2D((2, 2), strides=(2, 2), name='pool3')(conv3_1)

up2_2 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up22', padding="same")(conv3_1)
conv2_2 = concatenate([up2_2, c2], name='merge22', axis=3) #x10
conv2_2 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(conv2_2)
conv2_2 = Dropout(0.5)(conv2_2)
conv2_2 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(conv2_2)
conv2_2 = Dropout(0.5)(conv2_2)

up1_3 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up13', padding="same")(conv2_2)
conv1_3 = concatenate([up1_3, c1, c3], name='merge13', axis=3)
conv1_3 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(conv1_3)
conv1_3 = Dropout(0.5)(conv1_3)
conv1_3 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(conv1_3)
conv1_3 = Dropout(0.5)(conv1_3)

conv4_1 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(pool3)
conv4_1 = Dropout(0.5)(conv4_1)
conv4_1 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(conv4_1)
conv4_1 = Dropout(0.5)(conv4_1)
pool4 = MaxPooling2D((2, 2), strides=(2, 2), name='pool4')(conv4_1)

up3_2 = Conv2DTranspose(nb_filter[2], (2, 2), strides=(2, 2), name='up32', padding="same")(conv4_1)
conv3_2 = concatenate([up3_2, conv3_1], name='merge32', axis=3) #x20
conv3_2 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(conv3_2)
conv3_2 = Dropout(0.5)(conv3_2)
conv3_2 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(conv3_2)
conv3_2 = Dropout(0.5)(conv3_2)

up2_3 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up23', padding="same")(conv3_2)
conv2_3 = concatenate([up2_3, c2, conv2_2], name='merge23', axis=3)
conv2_3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(conv2_3)
conv2_3 = Dropout(0.5)(conv2_3)
conv2_3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(conv2_3)
conv2_3 = Dropout(0.5)(conv2_3)

up1_4 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up14', padding="same")(conv2_3)
conv1_4 = concatenate([up1_4, c1, c3, conv1_3], name='merge14', axis=3)
conv1_4 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(conv1_4)
conv1_4 = Dropout(0.5) (conv1_4)
conv1_4 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(conv1_4)
conv1_4 = Dropout(0.5)(conv1_4)

conv5_1 = Conv2D(512, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(pool4)
conv5_1 = Dropout(0.5) (conv5_1)
conv5_1 = Conv2D(512, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(conv5_1)
conv5_1 = Dropout(0.5)(conv5_1)

up4_2 = Conv2DTranspose(nb_filter[3], (2, 2), strides=(2, 2), name='up42', padding="same")(conv5_1)
conv4_2 = concatenate([up4_2, conv4_1], name='merge42', axis=3) #x30
conv4_2 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(conv4_2)
conv4_2 = Dropout(0.5)(conv4_2)
conv4_2 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(conv4_2)
conv4_2 = Dropout(0.5)(conv4_2)

up3_3 = Conv2DTranspose(nb_filter[2], (2, 2), strides=(2, 2), name='up33', padding="same")(conv4_2)
conv3_3 = concatenate([up3_3, conv3_1, conv3_2], name='merge33', axis=3)
conv3_3 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(conv3_3)
conv3_3 = Dropout(0.5)(conv3_3)
conv3_3 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(conv3_3)
conv3_3 = Dropout(0.5)(conv3_3)

up2_4 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up24', padding="same")(conv3_3)
conv2_4 = concatenate([up2_4, c2, conv2_2, conv2_3], name='merge24', axis=3)
conv2_4 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(conv2_4)
conv2_4 = Dropout(0.5)(conv2_4)
conv2_4 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(conv2_4)
conv2_4 = Dropout(0.5)(conv2_4)

up1_5 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up15', padding="same")(conv2_4)
conv1_5 = concatenate([up1_5, c1, c3, conv1_3, conv1_4], name='merge15', axis=3)
conv1_5 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(conv1_5)
conv1_5 = Dropout(0.5) (conv1_5)
conv1_5 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding="same")(conv1_5)
conv1_5 = Dropout(0.5)(conv1_5)

nestnet_output_4 = Conv2D(6, (1, 1), activation='sigmoid', kernel_initializer='he_normal', padding="same")(conv1_5)
model = Model([inputs], [nestnet_output_4])

In [ ]:
model.summary()

In [ ]:
weigths = [0.17649986,  7.78453571, 41.53978194, 65.20657672, 96.75504125,  6.40743063]

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR), loss=dice_coef_multilabelloss, metrics=[dice_coef_multilabel, tf.keras.metrics.MeanIoU(num_classes=6)])

In [ ]:
import keras

keras.utils.plot_model(model, "Uent++.png", show_shapes=True)

In [ ]:
history = model.fit(X_train, y_train_cat,
                    validation_data=(X_test, y_test_cat), 
                    batch_size = 32, 
                    verbose=1, 
                    epochs=50,  
                    shuffle=False)

In [ ]:
def training_plot(metrics, history):
    f, ax = plt.subplots(1, len(metrics), figsize=(5*len(metrics), 4))
    for idx, metric in enumerate(metrics):
        ax[idx].plot(history.history[metric], ls='dashed')
        ax[idx].set_xlabel("Epochs")
        ax[idx].set_ylabel(metric)
        ax[idx].plot(history.history['val_' + metric]);
        ax[idx].legend([metric, 'val_' + metric])

In [ ]:
training_plot(['loss', 'dice_coef_multilabel', 'mean_io_u_1'], history);

## Evaluation of the model

In [ ]:
Unet.save('Unet.hdf5')

In [ ]:
volume_img_1, volume_seg_1 = generate_patient_processed_data(cleaned_data["patient_paths"][12], cleaned_data["patient_segmentation"][12])

In [ ]:
transposed_volume_dcm = np.transpose(volume_img_1, (2, 0, 1))
transposed_volume_dcm = np.expand_dims(transposed_volume_dcm, axis=-1)
transposed_volume_dcm.shape

In [ ]:
transpose_volume_nii = np.transpose(volume_seg, (2, 0, 1))
transpose_volume_nii = np.expand_dims(transpose_volume_nii, axis=3)
transpose_volume_nii.shape

In [ ]:
y_pred=Unet.predict(transposed_volume_dcm)
y_pred_argmax=np.argmax(y_pred, axis=3)

In [ ]:
from keras.metrics import MeanIoU
n_classes = 6
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(transpose_volume_nii, y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())

### OVERFITTING... next train on complete patients